In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import nltk
import pickle
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn import svm


# **Loading data from Github repo**

In [3]:
names = ['id',	'name',	'host_id',	'host_name',	'neighbourhood_group',	'neighbourhood',	'latitude',	
         'longitude',	'room_type',	'price',	'minimum_nights',	'number_of_reviews',	'last_review',	
         'reviews_per_month',	'calculated_host_listings_count',	'availability_365']
df = pd.read_csv('https://raw.githubusercontent.com/lmxy0212/ML_project/master/new-york-city-airbnb-open-data/AB_NYC_2019.csv',
                names=names,na_values='?',header=None)
df = df.dropna()
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,6,7,9,10,11,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
1,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
2,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
4,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
5,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


# **Formating data to be np array**

In [4]:
airbnb_name = np.array(df['name'][1:])
reviews_per_month = np.array(df['reviews_per_month'][1:]).astype(np.float)
n = np.shape(airbnb_name)[0]
print(n)
print(airbnb_name)
print(np.array(reviews_per_month))
print(np.shape(airbnb_name))
print(np.shape(reviews_per_month))
print(np.sort(reviews_per_month)[-50])
print(np.mean(reviews_per_month), np.max(reviews_per_month), np.median(reviews_per_month))


38821
['Clean & quiet apt home by the park' 'Skylit Midtown Castle'
 'Cozy Entire Floor of Brownstone' ... 'Seas The Moment'
 '1B-1B apartment near by Metro' 'Cozy Private Room in Bushwick, Brooklyn']
[0.21 0.38 4.64 ... 1.   2.   1.  ]
(38821,)
(38821,)
11.16
1.3732291800829448 58.5 0.72


# **Creating bag of words for each airbnb name**

In [5]:
vectorizer = CountVectorizer()
bow = vectorizer.fit(airbnb_name)
bow = vectorizer.transform(airbnb_name)
bow

<38821x6961 sparse matrix of type '<class 'numpy.int64'>'
	with 222889 stored elements in Compressed Sparse Row format>

In [0]:
# print(bow)
# # (seq_num, feature_num)  count
# print(vectorizer.vocabulary_.get("great")) #feature number of "great"

In [6]:
X = bow.toarray()
print(X)
print(np.shape(X))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(38821, 6961)


# **Train Test Split**

In [7]:
Xtr, Xts, ytr, yts = train_test_split(X, reviews_per_month, test_size = 1/3)
print("Xtr:\n", Xtr, "\nXts:\n",Xts,"\nytr:\n", ytr, "\nyts:\n", yts)
print("\nXtr.shape:", Xtr.shape, "Xts.shape", Xts.shape)
print("\nytr.shape:", ytr.shape, "yts.shape:", yts.shape)
ytr = ytr.astype(np.float)
yts = yts.astype(np.float)

Xtr:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
Xts:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
ytr:
 [4.33 0.2  0.52 ... 0.27 0.16 0.72] 
yts:
 [1.   1.   1.26 ... 2.83 0.02 1.81]

Xtr.shape: (25880, 6961) Xts.shape (12941, 6961)

ytr.shape: (25880,) yts.shape: (12941,)


## **Using linear regression to fit the data and saving in linear_reg.p**

In [0]:
regr = linear_model.LinearRegression()
regr.fit(Xtr,ytr)
print(regr.intercept_)
print(regr.coef_)


NameError: ignored

In [0]:
with open( "linear_reg.p", "wb" ) as fp:
    pickle.dump( [regr, Xtr, ytr, Xts, yts],  fp)

In [0]:
with open( "linear_reg.p", "rb" ) as fp:
    regr, Xtr, ytr, Xts, yts = pickle.load(fp)
ytr_pred = regr.predict(Xtr)
lossm = np.linalg.norm(ytr_pred - ytr)**2
lossm = (1/n) * np.linalg.norm(ytr_pred - ytr)**2
print("Average loss on training data:",lossm)
yts_pred = regr.predict(Xts)
lossm = (1/n) * np.linalg.norm(yts_pred - yts)**2
print("Average loss on test data:", lossm)


Average loss on training data: 1.2994240550299205
Average loss on test data: 9.864150215880753e+23


# **Using Binary Classification**

In [0]:
# if review_per_month > 10, we classify the airbnb as popular
ytr = (ytr > 10).astype(int)
yts = (yts > 10).astype(int)

## **Logistics Regression**

In [9]:
logreg = linear_model.LogisticRegression(verbose=10, solver='lbfgs',\
                                         multi_class='multinomial',max_iter=500)
logreg.fit(Xtr,ytr)
with open( "logreg.p", "wb" ) as fp:
    pickle.dump( [logreg, Xtr, ytr, Xts, yts],  fp)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   27.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   27.3s finished


In [16]:
with open( "logreg.p", "rb" ) as fp:
    logreg, Xtr, ytr, Xts, yts = pickle.load(fp)

yhat_zeros = np.zeros(np.shape(Xtr)[0])
acc_zeros = np.mean(yhat_zeros == ytr)
print('Accuaracy for all zeros for training set = {0:f}'.format(acc_zeros))

yhat_tr = logreg.predict(Xtr)
acc_tr = np.mean(yhat_tr == ytr)
print('Accuaracy on training set = {0:f}'.format(acc_tr))

yhat_zeros = np.zeros(np.shape(Xts)[0])
acc_zeros = np.mean(yhat_zeros == yts)
print('Accuaracy for all zeros for test set = {0:f}'.format(acc_zeros))

yhat = logreg.predict(Xts)
acc = np.mean(yhat == yts)
print('Accuaracy on test set = {0:f}'.format(acc))

Accuaracy for all zeros for training set = 0.998223
Accuaracy on training set = 0.998532
Accuaracy for all zeros for test set = 0.997295
Accuaracy on test set = 0.997450


## **SVM**

In [21]:
svc = svm.SVC(probability=False,  kernel="rbf",verbose=10)
svc.fit(Xtr,ytr)
with open( "svm.p", "wb" ) as fp:
    pickle.dump( [svc, Xtr, ytr, Xts, yts],  fp)

[LibSVM]

In [22]:
with open( "svm.p", "rb" ) as fp:
    svc, Xtr, ytr, Xts, yts = pickle.load(fp)

yhat_zeros = np.zeros(np.shape(Xtr)[0])
acc_zeros = np.mean(yhat_zeros == ytr)
print('Accuaracy for all zeros for training set = {0:f}'.format(acc_zeros))

yhat_tr = svc.predict(Xtr)
acc_tr = np.mean(yhat_tr == ytr)
print('Accuaracy on training set = {0:f}'.format(acc_tr))

yhat_zeros = np.zeros(np.shape(Xts)[0])
acc_zeros = np.mean(yhat_zeros == yts)
print('Accuaracy for all zeros for test set = {0:f}'.format(acc_zeros))

yhat = svc.predict(Xts)
acc = np.mean(yhat == yts)
print('Accuaracy on test set = {0:f}'.format(acc))

Accuaracy for all zeros for training set = 0.998223
Accuaracy on training set = 0.998377
Accuaracy for all zeros for test set = 0.997295
Accuaracy on test set = 0.997373


## **Neural Network**

In [0]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model

K.clear_session()

In [33]:
model = Sequential()
model.add(Dense(units=4, input_shape=(np.shape(Xtr)[1],), activation='sigmoid', name='hidden'))
model.add(Dense(units=1, activation='sigmoid', name='output'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 4)                 27848     
_________________________________________________________________
output (Dense)               (None, 1)                 5         
Total params: 27,853
Trainable params: 27,853
Non-trainable params: 0
_________________________________________________________________


In [44]:
opt = optimizers.Adam(lr=0.01)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])
hist = model.fit(Xtr, ytr, epochs=100, batch_size=100)
model.save("NN_mod.h5")


Epoch 1/100
259/259 [==============================] - 1s 5ms/step - loss: 0.0028 - accuracy: 0.9990
Epoch 2/100
259/259 [==============================] - 1s 5ms/step - loss: 0.0023 - accuracy: 0.9993
Epoch 3/100
259/259 [==============================] - 1s 5ms/step - loss: 0.0023 - accuracy: 0.9992
Epoch 4/100
259/259 [==============================] - 1s 5ms/step - loss: 0.0021 - accuracy: 0.9993
Epoch 5/100
259/259 [==============================] - 1s 5ms/step - loss: 0.0020 - accuracy: 0.9992
Epoch 6/100
259/259 [==============================] - 1s 5ms/step - loss: 0.0019 - accuracy: 0.9993
Epoch 7/100
259/259 [==============================] - 1s 5ms/step - loss: 0.0018 - accuracy: 0.9993
Epoch 8/100
259/259 [==============================] - 1s 5ms/step - loss: 0.0017 - accuracy: 0.9993
Epoch 9/100
259/259 [==============================] - 1s 5ms/step - loss: 0.0017 - accuracy: 0.9994
Epoch 10/100
259/259 [==============================] - 1s 5ms/step - loss: 0.0016 - accura

In [45]:
model = load_model("NN_mod.h5")
# yhat_tr = model.predict(Xtr)
score, acc_tr = model.evaluate(Xtr, ytr, verbose=1)
print("Accuaracy on training set = %f" % acc_tr)

# yhat = model.predict(Xts)
score, acc = model.evaluate(Xts, yts, verbose=1)
print("Accuaracy on test set = %f" % acc)


809/809 [==============================] - 1s 2ms/step - loss: 2.6152e-04 - accuracy: 0.9998
Accuaracy on training set = 0.999845
405/405 [==============================] - 1s 2ms/step - loss: 0.0463 - accuracy: 0.9946
Accuaracy on test set = 0.994591
